### 1.Install Library(module) yang dibutuhkan

In [2]:
# %pip install tweepy
# %pip install Sastrawi
# %pip install numpy
# %pip install sklearn
# %pip install textblob
# %pip install pandas
# %pip install matplotlib

In [3]:
import tweepy

BEARERTOKEN = 'AAAAAAAAAAAAAAAAAAAAAJQrYQEAAAAA1BtlexYIrHRZvqkz6TEOraEC%2B0k%3D5cOp6U7ytEMpcTDctDQWVl6HFd0Y28pxhDnTGyHA9KiiC1MVCF'

client = tweepy.Client(bearer_token=BEARERTOKEN)

query = '"Jokowi 3 priode" lang:id'

# get data max 100
responses = client.search_recent_tweets(query=query, max_results=100, tweet_fields=['created_at'])
tweet100 = [tweet for tweet in responses.data]

# get max 1000
tweettexts = [tweet.text for tweet in tweepy.Paginator(client.search_recent_tweets, query=query, max_results=100).flatten(limit=1000)]

In [4]:
import pandas as pd
import numpy as np
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import string
import re

In [5]:
data = {
    'text' : tweettexts
}
df = pd.DataFrame(data)
df.to_csv(f"crawling.csv", mode='w', index=False, header=data.keys())

In [6]:
import pandas as pd

df = pd.read_csv('crawling.csv', encoding='unicode_escape')
df.head(5)

,text
0,@Nandaa_80 @sandiuno Tugasnya mempromosikan jo...
1,RT @yani_akhmad: @geloraco Rencana buat Pilpre...
2,@geloraco Rencana buat Pilpres 2024 Jokowi 3 p...
3,"RT @SakiinaQhiCO_33: Sebuah kewajaran dan sah""..."
4,"RT @SakiinaQhiCO_33: Sebuah kewajaran dan sah""..."


In [7]:
#Cleaning Text
def textcleaner(text):
    new_text = str(text)
    
    # remove old style retweet text "RT"
    new_text = re.sub(r'^RT\s+', '', new_text)
    
    # remove username
    new_text = re.sub(r'@([A-Za-z0-9_]+)', '', new_text)

    # remove hyperlinks
    new_text = re.sub(r'https?:\/\/.*[\r\n]*', '', new_text)

    # remove invalid character
    new_text = re.sub("[^A-Za-z" "]+", " ", new_text)
    
    new_text    = re.sub(r'\b\w(1,2)\b',' ', new_text) #menghilangkan 2 kata
    
    return new_text

In [8]:
def caselower(sentences):
    return sentences.lower()

In [9]:
def tokenmaker(sentences):
    return sentences.strip().split()

In [10]:
#Stop Removal
stopword = nltk.corpus.stopwords.words("indonesian")
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

In [11]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(words):
    return [stemmer.stem(word) for word in words]

In [12]:
# create labeler

pos_kata = [pos.strip() for pos in open('kata_positif.txt', 'r').readlines()]
neg_kata = [neg.strip() for neg in open('kata_negatif.txt', 'r').readlines()]

#list kata-kata negasi
negasi = [neg.strip() for neg in open('negasi.txt', 'r').readlines()]


def labeling(stemmed):
    count_pos = 0
    count_neg = 0
    for ind, stem in enumerate(stemmed):
        if stem.strip() in pos_kata:
            if stemmed[ind-1].strip() in negasi:
                count_neg += 1
            else:
                count_pos += 1
        elif stem.strip() in neg_kata:
            if stemmed[ind-1].strip() in negasi:
                count_pos += 1
            else:
                count_neg += 1

    return 'NETRAL' if count_pos == count_neg else 'POSITIF' if count_pos > count_neg else 'NEGATIF'
    


In [13]:
# cleaning text
df['clean_text'] = df['text'].apply(lambda x : textcleaner(x))

In [14]:
# cover ke lower case
df['lower'] = df['clean_text'].apply(lambda x : caselower(x))

In [15]:
# tokenizing
df['token'] = df['lower'].apply(lambda x : tokenmaker(x))

In [16]:
# stopword removing
df["stop_removed"] = df['token'].apply(lambda x: remove_stopwords(x))

In [17]:
# stemming
df["stemmed"] = df['stop_removed'].apply(lambda x: stemming(x))
print("Stemming selesai!")

Stemming selesai!


In [ ]:
# labelling sentiment
df["sentiment"] = df['stemmed'].apply(lambda x: labeling(x))
df.to_csv(f"labeled.csv", mode='w', index=False, header=False)
df.head(df.size)